## Import Library Yang Diperlukan

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ast
from google.colab import files
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle (1).json': b'{"username":"zahidfaqih","key":"3491bc2a15d3934ff51f6174c958647c"}'}

## Loading Data

In [3]:
!kaggle datasets download -d tmdb/tmdb-movie-metadata

Dataset URL: https://www.kaggle.com/datasets/tmdb/tmdb-movie-metadata
License(s): other
tmdb-movie-metadata.zip: Skipping, found more recently modified local copy (use --force to force download)


In [4]:
!unzip tmdb-movie-metadata.zip

Archive:  tmdb-movie-metadata.zip
replace tmdb_5000_credits.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: tmdb_5000_credits.csv   
replace tmdb_5000_movies.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: tmdb_5000_movies.csv    


In [5]:
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

## Data Understanding

Pertama-tama lihat dulu info dari masing-masing dataset

In [6]:
credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   movie_id  4803 non-null   int64 
 1   title     4803 non-null   object
 2   cast      4803 non-null   object
 3   crew      4803 non-null   object
dtypes: int64(1), object(3)
memory usage: 150.2+ KB


In [7]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

dataset movie dan credits kita gabung terlebih dahulu

In [8]:
movies = movies.merge(credits)

In [9]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4809 entries, 0 to 4808
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4809 non-null   int64  
 1   genres                4809 non-null   object 
 2   homepage              1713 non-null   object 
 3   id                    4809 non-null   int64  
 4   keywords              4809 non-null   object 
 5   original_language     4809 non-null   object 
 6   original_title        4809 non-null   object 
 7   overview              4806 non-null   object 
 8   popularity            4809 non-null   float64
 9   production_companies  4809 non-null   object 
 10  production_countries  4809 non-null   object 
 11  release_date          4808 non-null   object 
 12  revenue               4809 non-null   int64  
 13  runtime               4807 non-null   float64
 14  spoken_languages      4809 non-null   object 
 15  status               

In [10]:
movies.isnull().sum()

,0
budget,0
genres,0
homepage,3096
id,0
keywords,0
original_language,0
original_title,0
overview,3
popularity,0
production_companies,0


In [11]:
movies.duplicated().sum()

0

In [12]:
jumlah_baris, jumlah_kolom= movies.shape
print("Jumlah baris:", jumlah_baris)
print("Jumlah kolom:", jumlah_kolom)

Jumlah baris: 4809
Jumlah kolom: 23


In [13]:
movies.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,285,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,206647,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,49026,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,49529,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


## Data Preprocessing

Dengan kata lain, agar sebuah film dapat tampil di tangga film, film tersebut harus memiliki lebih banyak suara daripada setidaknya 90% film dalam daftar.

In [14]:
m= movies['vote_count'].quantile(0.9)

C = movies['vote_average'].mean()

# Membuat kolom Weighted Rating
def weighted_rating(x, C, m):
    v = x['vote_count']
    R = x['vote_average']
    return (v * R + C * m) / (v + m)

movies['weighted_rating'] = movies.apply(weighted_rating, axis=1, C=C, m=m)


In [15]:
movies.sort_values('weighted_rating', ascending=False).head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew,weighted_rating
1883,25000000,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 80, ""name...",NaN,278,"[{""id"": 378, ""name"": ""prison""}, {""id"": 417, ""n...",en,The Shawshank Redemption,Framed in the 1940s for the double murder of h...,136.747729,"[{""name"": ""Castle Rock Entertainment"", ""id"": 97}]",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Fear can hold you prisoner. Hope can set you f...,The Shawshank Redemption,8.5,8205,278,"[{""cast_id"": 3, ""character"": ""Andy Dufresne"", ...","[{""credit_id"": ""52fe4231c3a36847f800b153"", ""de...",8.058576
662,63000000,"[{""id"": 18, ""name"": ""Drama""}]",http://www.foxmovies.com/movies/fight-club,550,"[{""id"": 825, ""name"": ""support group""}, {""id"": ...",en,Fight Club,A ticking-time-bomb insomniac and a slippery s...,146.757391,"[{""name"": ""Regency Enterprises"", ""id"": 508}, {...",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Mischief. Mayhem. Soap.,Fight Club,8.3,9413,550,"[{""cast_id"": 4, ""character"": ""The Narrator"", ""...","[{""credit_id"": ""55731b8192514111610027d7"", ""de...",7.938689
65,185000000,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 28, ""name...",http://thedarkknight.warnerbros.com/dvdsite/,155,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight,Batman raises the stakes in his war on crime. ...,187.322927,"[{""name"": ""DC Comics"", ""id"": 429}, {""name"": ""L...",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Why So Serious?,The Dark Knight,8.2,12002,155,"[{""cast_id"": 35, ""character"": ""Bruce Wayne"", ""...","[{""credit_id"": ""55a0eb4a925141296b0010f8"", ""de...",7.919564
3235,8000000,"[{""id"": 53, ""name"": ""Thriller""}, {""id"": 80, ""n...",NaN,680,"[{""id"": 396, ""name"": ""transporter""}, {""id"": 14...",en,Pulp Fiction,"A burger-loving hit man, his philosophical par...",121.463076,"[{""name"": ""Miramax Films"", ""id"": 14}, {""name"":...",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Just because you are a character doesn't mean ...,Pulp Fiction,8.3,8428,680,"[{""cast_id"": 2, ""character"": ""Vincent Vega"", ""...","[{""credit_id"": ""52fe4269c3a36847f801caa3"", ""de...",7.904036
96,160000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 53, ""nam...",http://inceptionmovie.warnerbros.com/,27205,"[{""id"": 1014, ""name"": ""loss of lover""}, {""id"":...",en,Inception,"Cobb, a skilled thief who commits corporate es...",167.583710,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Your mind is the scene of the crime.,Inception,8.1,13752,27205,"[{""cast_id"": 1, ""character"": ""Dom Cobb"", ""cred...","[{""credit_id"": ""56e8462cc3a368408400354c"", ""de...",7.862848


Untuk content-based filtering variabel yang digunakan adalah id, title, keywords, genres, cast, dan overview

In [16]:
movies = movies[['id','title','keywords','genres','cast','overview','weighted_rating']]

In [17]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4809 entries, 0 to 4808
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               4809 non-null   int64  
 1   title            4809 non-null   object 
 2   keywords         4809 non-null   object 
 3   genres           4809 non-null   object 
 4   cast             4809 non-null   object 
 5   overview         4806 non-null   object 
 6   weighted_rating  4809 non-null   float64
dtypes: float64(1), int64(1), object(5)
memory usage: 263.1+ KB


In [18]:
movies.head()

,id,title,keywords,genres,cast,overview,weighted_rating
0,19995,Avatar,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","In the 22nd century, a paraplegic Marine is di...",7.050449
1,285,Pirates of the Caribbean: At World's End,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","Captain Barbossa, long believed to be dead, ha...",6.665452
2,206647,Spectre,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...",A cryptic message from Bond’s past sends him o...,6.239407
3,49026,The Dark Knight Rises,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...","[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...",Following the death of District Attorney Harve...,7.346343
4,49529,John Carter,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","John Carter is a war-weary, former military ca...",6.096523


dikarenakan variabel keywords, gennres, cast berbentuk JSON yang berisi id dan nama, dan kita hanya butuh namanya saja, maka kita filter variabelnya

In [19]:
def extract_names(json_str):
    try:
        keywords = ast.literal_eval(json_str)
        return ",".join([item['name'] for item in keywords])
    except (ValueError, SyntaxError):
        return ""

movies['genres'] = movies['genres'].apply(extract_names)
movies['keywords'] = movies['keywords'].apply(extract_names)
movies['cast'] = movies['cast'].apply(extract_names)

In [20]:
movies.head()

,id,title,keywords,genres,cast,overview,weighted_rating
0,19995,Avatar,"culture clash,future,space war,space colony,so...","Action,Adventure,Fantasy,Science Fiction","Sam Worthington,Zoe Saldana,Sigourney Weaver,S...","In the 22nd century, a paraplegic Marine is di...",7.050449
1,285,Pirates of the Caribbean: At World's End,"ocean,drug abuse,exotic island,east india trad...","Adventure,Fantasy,Action","Johnny Depp,Orlando Bloom,Keira Knightley,Stel...","Captain Barbossa, long believed to be dead, ha...",6.665452
2,206647,Spectre,"spy,based on novel,secret agent,sequel,mi6,bri...","Action,Adventure,Crime","Daniel Craig,Christoph Waltz,Léa Seydoux,Ralph...",A cryptic message from Bond’s past sends him o...,6.239407
3,49026,The Dark Knight Rises,"dc comics,crime fighter,terrorist,secret ident...","Action,Crime,Drama,Thriller","Christian Bale,Michael Caine,Gary Oldman,Anne ...",Following the death of District Attorney Harve...,7.346343
4,49529,John Carter,"based on novel,mars,medallion,space travel,pri...","Action,Adventure,Science Fiction","Taylor Kitsch,Lynn Collins,Samantha Morton,Wil...","John Carter is a war-weary, former military ca...",6.096523


dikarenakan pada fungsi extract_names, jika terjadi error saat memproses string JSON (misalnya, format JSON tidak valid), fungsi ini akan mengembalikan string kosong (""), maka kita rubah nilai string kosong menjadi nan agar terdeteksi bahwa string tersebut merupakan missing value dengan fungsi replace().

In [21]:
movies = movies.replace('', np.nan)
movies.isnull().sum()

,0
id,0
title,0
keywords,412
genres,28
cast,43
overview,3
weighted_rating,0


terdapat missing value maka kita hapus dengan fungsi dropna()

In [22]:
movies=movies.dropna()
movies.isnull().sum()

,0
id,0
title,0
keywords,0
genres,0
cast,0
overview,0
weighted_rating,0


In [23]:
import itertools

all_genres = list(itertools.chain(*movies['genres'].str.split(',').apply(lambda x: [i.strip() for i in x])))

all_keywords = list(itertools.chain(*movies['keywords'].str.split(',').apply(lambda x: [i.strip() for i in x])))

all_cast = list(itertools.chain(*movies['cast'].str.split(',').apply(lambda x: [i.strip() for i in x])))
# Filter untuk mengecualikan string kosong
unique_genres = set(genre for genre in all_genres if genre)

unique_keywords = set(keyword for keyword in all_keywords if keyword)

unique_cast = set(cast for cast in all_cast if cast)

print("Jumlah genre unik :",len(unique_genres))
print("Jumlah keywords unik :",len(unique_keywords))
print("Jumlah cast unik :",len(unique_cast))

Jumlah genre unik : 20
Jumlah keywords unik : 9799
Jumlah cast unik : 52461


membuang spasi pada cast agar nama seperti taylor berbeda antara taylor swift dan taylor kitsch

In [24]:
movies['cast'] = movies['cast'].str.replace(' ', '')

menggabungkan semua variabel ke 1 variabel features

In [44]:
movies['features'] = ' ' + movies['genres'] + ' ' + movies['keywords'] + ' ' + movies['cast'] + ' ' + movies['overview'] + movies['weighted_rating'].astype(str)

lowercase text dan mengganti koma dengan spasi

In [45]:
movies['features'] = movies['features'].str.lower()
movies['features'] = movies['features'].str.replace(',', ' ')

In [46]:
movies.head()

,id,title,keywords,genres,cast,overview,weighted_rating,features,embedding
0,19995,Avatar,"culture clash,future,space war,space colony,so...","Action,Adventure,Fantasy,Science Fiction","SamWorthington,ZoeSaldana,SigourneyWeaver,Step...","In the 22nd century, a paraplegic Marine is di...",7.050449,action adventure fantasy science fiction cult...,"[-0.060945015, 0.10030734, 0.047473032, 0.0536..."
1,285,Pirates of the Caribbean: At World's End,"ocean,drug abuse,exotic island,east india trad...","Adventure,Fantasy,Action","JohnnyDepp,OrlandoBloom,KeiraKnightley,Stellan...","Captain Barbossa, long believed to be dead, ha...",6.665452,adventure fantasy action ocean drug abuse exo...,"[-0.06520977, 0.09566249, 0.046125982, 0.05581..."
2,206647,Spectre,"spy,based on novel,secret agent,sequel,mi6,bri...","Action,Adventure,Crime","DanielCraig,ChristophWaltz,LéaSeydoux,RalphFie...",A cryptic message from Bond’s past sends him o...,6.239407,action adventure crime spy based on novel sec...,"[-0.063608505, 0.103192866, 0.047809966, 0.056..."
3,49026,The Dark Knight Rises,"dc comics,crime fighter,terrorist,secret ident...","Action,Crime,Drama,Thriller","ChristianBale,MichaelCaine,GaryOldman,AnneHath...",Following the death of District Attorney Harve...,7.346343,action crime drama thriller dc comics crime f...,"[-0.061474945, 0.100273065, 0.04702213, 0.0505..."
4,49529,John Carter,"based on novel,mars,medallion,space travel,pri...","Action,Adventure,Science Fiction","TaylorKitsch,LynnCollins,SamanthaMorton,Willem...","John Carter is a war-weary, former military ca...",6.096523,action adventure science fiction based on nov...,"[-0.06361959, 0.09749846, 0.04613337, 0.053425..."


## Modelling

### Modelling dengan TF-IDF

In [47]:
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(movies['features'])

In [48]:
tfidf_matrix.shape

(4384, 79116)

Perhatikanlah, matriks yang kita miliki berukuran (4384, 79116). Nilai 4384 merupakan ukuran data dan 79116 merupakan matrik features dari semua film.




In [49]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [50]:
cosine_sim

array([[1.        , 0.01150599, 0.00424635, ..., 0.00800514, 0.01180569,
        0.00363262],
       [0.01150599, 1.        , 0.01887073, ..., 0.01873573, 0.02137729,
        0.01038971],
       [0.00424635, 0.01887073, 1.        , ..., 0.01854263, 0.01300524,
        0.00654078],
       ...,
       [0.00800514, 0.01873573, 0.01854263, ..., 1.        , 0.01744442,
        0.02373094],
       [0.01180569, 0.02137729, 0.01300524, ..., 0.01744442, 1.        ,
        0.03227221],
       [0.00363262, 0.01038971, 0.00654078, ..., 0.02373094, 0.03227221,
        1.        ]])

In [70]:
# Membuat dataframe dari variabel cosine_sim
cosine_sim_df = pd.DataFrame(cosine_sim, index=movies['title'], columns=movies['title'])
print('Shape:', cosine_sim_df.shape)


Shape: (4384, 4384)


In [71]:
def movies_recommendation(title, similarity_data=cosine_sim_df, items=movies[['title', 'genres', 'cast', 'overview', 'keywords','weighted_rating']], k=10):


    # Mengambil data dengan menggunakan argpartition untuk melakukan partisi secara tidak langsung sepanjang sumbu yang diberikan
    # Dataframe diubah menjadi numpy
    # Range(start, stop, step)
    index = similarity_data.loc[:,title].to_numpy().argpartition(
        range(-1, -k, -1))

    # Mengambil data dengan similarity terbesar dari index yang ada
    closest = similarity_data.columns[index[-1:-(k+2):-1]]

    closest = closest.drop(title, errors='ignore')

    return pd.DataFrame(closest).merge(items).head(k)

In [96]:
movies_recommendation("Pirates of the Caribbean: At World's End")

,title,genres,cast,overview,keywords,weighted_rating
0,Pirates of the Caribbean: Dead Man's Chest,"Adventure,Fantasy,Action","JohnnyDepp,OrlandoBloom,KeiraKnightley,Stellan...",Captain Jack Sparrow works his way out of a bl...,"witch,fortune teller,bondage,exotic island,mon...",6.764147
1,Pirates of the Caribbean: The Curse of the Bla...,"Adventure,Fantasy,Action","JohnnyDepp,GeoffreyRush,OrlandoBloom,KeiraKnig...","Jack Sparrow, a freewheeling 17th-century pira...","exotic island,blacksmith,east india trading co...",7.206264
2,Pirates of the Caribbean: On Stranger Tides,"Adventure,Action,Fantasy","JohnnyDepp,PenélopeCruz,IanMcShane,KevinMcNall...",Captain Jack Sparrow crosses paths with a woma...,"sea,captain,mutiny,sword,prime minister,sailin...",6.316578
3,The Pirates! In an Adventure with Scientists!,"Animation,Adventure,Family,Comedy","HughGrant,BrendanGleeson,JeremyPiven,BrianBles...",The luxuriantly bearded Pirate Captain is a bo...,"rivalry,stop motion,pirate,aftercreditsstinger...",6.144405
4,Anna and the King,"Drama,History,Romance","JodieFoster,ChowYun-fat,BaiLing,TomFelton,Rand...",The story of the romance between the King of S...,"civil war,father son relationship,east india t...",6.122219
5,The Blue Lagoon,"Romance,Adventure,Drama","BrookeShields,ChristopherAtkins,LeoMcKern,Will...",Two small children and a ship's cook survive a...,"sexual identity,shipwreck,lovers,stranded,maro...",6.039476
6,In the Heart of the Sea,"Thriller,Drama,Adventure,Action,History","ChrisHemsworth,BenjaminWalker,CillianMurphy,Br...","In the winter of 1820, the New England whaling...","suicide,ocean,sea,hunger,shipwreck,ship,whale,...",6.259262
7,Cutthroat Island,"Action,Adventure","GeenaDavis,MatthewModine,FrankLangella,MauryCh...","Morgan Adams and her slave, William Shaw, are ...","exotic island,treasure,map,ship,scalp,pirate",6.065529
8,The Pirate,"Music,Romance","JudyGarland,GeneKelly,WalterSlezak,GladysCoope...","A girl is engaged to the local richman, but me...","musical,pirate",6.097154
9,Swept Away,"Comedy,Romance","Madonna,AdrianoGiannini,BruceGreenwood,Elizabe...",Stranded and alone on a desert island during a...,"adultery,island,shipwreck,tropical island",6.058360


### Modelling dengan word2vec

In [77]:
movies['tokens'] = movies['features'].apply(lambda x: x.split())

In [78]:
from gensim.models import Word2Vec
model = Word2Vec(sentences=movies['tokens'], vector_size=100, window=5, min_count=1, workers=4, sg=1)


In [79]:
def get_average_vector(tokens, model, vector_size):
    vectors = [model.wv[word] for word in tokens if word in model.wv]
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(vector_size)
movies['embedding'] = movies['tokens'].apply(lambda x: get_average_vector(x, model, 100))


In [80]:
embeddings = np.array(movies['embedding'].tolist())
similarity_matrix = cosine_similarity(embeddings)

In [81]:
similarity_matrix

array([[0.99999994, 0.98201907, 0.98346996, ..., 0.9428194 , 0.950506  ,
        0.9321596 ],
       [0.98201907, 1.        , 0.9929674 , ..., 0.9783201 , 0.9796182 ,
        0.9764613 ],
       [0.98346996, 0.9929674 , 0.9999999 , ..., 0.97537076, 0.9740039 ,
        0.96768713],
       ...,
       [0.9428194 , 0.9783201 , 0.97537076, ..., 0.9999997 , 0.9922736 ,
        0.9923089 ],
       [0.950506  , 0.9796182 , 0.9740039 , ..., 0.9922736 , 0.9999998 ,
        0.9907431 ],
       [0.9321596 , 0.9764613 , 0.96768713, ..., 0.9923089 , 0.9907431 ,
        1.0000001 ]], dtype=float32)

In [88]:
sim_df_w2v = pd.DataFrame(similarity_matrix, index=movies['title'], columns=movies['title'])
print('Shape:', sim_df_w2v.shape)

sim_df_w2v.sample(5, axis=1).sample(10, axis=0)

Shape: (4384, 4384)


title,Holy Motors,The Vow,Fear Clinic,Four Christmases,Top Cat Begins
title,,,,,
The Love Guru,0.994265,0.993370,0.994189,0.994695,0.987173
Grave Encounters,0.990634,0.984049,0.987504,0.985176,0.976089
Eden,0.995494,0.985748,0.988793,0.991086,0.982796
Clockwatchers,0.985254,0.987225,0.985572,0.984985,0.974229
Bowfinger,0.994142,0.990570,0.993512,0.992535,0.985007
Love & Basketball,0.991971,0.991548,0.987371,0.996062,0.988651
Sorority Boys,0.995874,0.990825,0.990691,0.994758,0.986493
Kicks,0.995214,0.992967,0.994999,0.993665,0.988563
The Intern,0.992491,0.988724,0.995961,0.991925,0.984681


In [89]:
def movies_recommendation_embed(title, similarity_data=sim_df_w2v, items=movies[['title', 'genres', 'cast', 'overview', 'keywords','weighted_rating']], k=15):
    # Mengambil data dengan menggunakan argpartition untuk melakukan partisi secara tidak langsung sepanjang sumbu yang diberikan
    # Dataframe diubah menjadi numpy
    # Range(start, stop, step)
    index = similarity_data.loc[:,title].to_numpy().argpartition(
        range(-1, -k, -1))

    # Mengambil data dengan similarity terbesar dari index yang ada
    closest = similarity_data.columns[index[-1:-(k+2):-1]]

    closest = closest.drop(title, errors='ignore')

    return pd.DataFrame(closest).merge(items).head(k)

In [97]:
movies_recommendation_embed("Pirates of the Caribbean: At World's End")

,title,genres,cast,overview,keywords,weighted_rating
0,Pirates of the Caribbean: The Curse of the Bla...,"Adventure,Fantasy,Action","JohnnyDepp,GeoffreyRush,OrlandoBloom,KeiraKnig...","Jack Sparrow, a freewheeling 17th-century pira...","exotic island,blacksmith,east india trading co...",7.206264
1,The Tourist,"Action,Thriller,Romance","JohnnyDepp,AngelinaJolie,PaulBettany,TimothyDa...",American tourist Frank (Johnny Depp) meets mys...,"paris,hotel,false identity,undercover agent,ro...",6.048128
2,New Nightmare,"Horror,Thriller,Mystery,Fantasy","HeatherLangenkamp,RobertEnglund,MikoHughes,Wes...",Freddy's back … and he's badder than ever! Nan...,"fire,kidnapping,nightmare,earthquake,insomnia,...",6.134460
3,Kung Fu Panda,"Adventure,Animation,Family,Comedy","JackBlack,DustinHoffman,AngelinaJolie,JackieCh...","When the Valley of Peace is threatened, lazy P...","china,martial arts,kung fu,mentor,snake,restau...",6.601726
4,Groundhog Day,"Romance,Fantasy,Drama,Comedy","BillMurray,AndieMacDowell,ChrisElliott,Stephen...","A narcissistic TV weatherman, along with his a...","deja vu,groundhog,weather forecast,telecaster,...",6.818650
5,The Huntsman: Winter's War,"Action,Adventure,Drama","ChrisHemsworth,CharlizeTheron,EmilyBlunt,Jessi...",As two evil sisters prepare to conquer the lan...,"witch,magic,fairy tale,snow white,huntsman",6.050645
6,Click,"Comedy,Drama,Fantasy,Romance","AdamSandler,KateBeckinsale,ChristopherWalken,D...",A workaholic architect finds a universal remot...,"regret,workaholic,heart attack,architect,dying...",6.043188
7,Indiana Jones and the Last Crusade,"Adventure,Action","HarrisonFord,SeanConnery,DenholmElliott,Alison...",When Dr. Henry Jones Sr. suddenly goes missing...,"saving the world,venice,holy grail,library,rid...",7.043937
8,Robin Hood,"Action,Adventure","RussellCrowe,CateBlanchett,MaxvonSydow,William...",When soldier Robin happens upon the dying Robe...,"robin hood,archer,knight,sherwood forest,bow a...",6.138889
9,Underworld,"Fantasy,Action,Thriller","KateBeckinsale,ScottSpeedman,MichaelSheen,Shan...",Vampires and werewolves have waged a nocturnal...,"budapest,subway,love of one's life,vampire,bit...",6.385290


## Evaluate

evaluasi dengan precision@k

In [98]:
def precision_at_k(recommended_items, relevant_items, k):
    recommended_at_k = recommended_items[:k]
    num_relevant = len(set(recommended_at_k) & set(relevant_items))
    return num_relevant / k

recommended_movies = movies_recommendation("Pirates of the Caribbean: At World's End")['title'].tolist()  # Mendapatkan daftar film yang direkomendasikan
relevant_movies = ['Pirates of the Caribbean: Dead Man\'s Chest', 'Pirates of the Caribbean: The Curse of the Black Pearl','Pirates of the Caribbean: On Stranger Tides',
                   'Pirates of the Caribbean: Dead Men Tell No Tales','Gone in Sixty Seconds','Spider-Man 3',
                   'Shrek the Third', 'Men in Black', 'Harry Potter and the Order of the Phoenix','Ratatouille']  # Daftar film yang relevan
k = 10  # Jumlah item teratas yang dipertimbangkan

precision = precision_at_k(recommended_movies, relevant_movies, k)
print(f"Precision@{k}: {precision}")

Precision@10: 0.3


In [100]:
def precision_at_k(recommended_items, relevant_items, k):
    recommended_at_k = recommended_items[:k]
    num_relevant = len(set(recommended_at_k) & set(relevant_items))
    return num_relevant / k

recommended_movies_embed = movies_recommendation_embed("Pirates of the Caribbean: At World's End")['title'].tolist()  # Mendapatkan daftar film yang direkomendasikan
relevant_movies = ['Pirates of the Caribbean: Dead Man\'s Chest', 'Pirates of the Caribbean: The Curse of the Black Pearl','Pirates of the Caribbean: On Stranger Tides',
                   'Pirates of the Caribbean: Dead Men Tell No Tales','Gone in Sixty Seconds','Spider-Man 3',
                   'Shrek the Third', 'Men in Black', 'Harry Potter and the Order of the Phoenix','Ratatouille']  # Daftar film yang relevan
k = 10  # Jumlah item teratas yang dipertimbangkan

precision = precision_at_k(recommended_movies_embed, relevant_movies, k)
print(f"Precision@{k}: {precision}")

Precision@10: 0.1
